In [9]:
import os
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
anime = pd.read_csv('C:\Users\Admin\Downloads\anime.csv')
des = pd.read_csv('anime_with_synopsis.csv')
anime.head(5)
des.head(5)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (569113892.py, line 1)

+**merge anime vs des**
+**keep imp col**

In [43]:
anime = anime.merge(des,on=['MAL_ID','Name','Score','Genres'])
anime = anime[['MAL_ID','Name','Score','Genres','Type','Episodes','Premiered','sypnopsis']]
anime.head(5)

,MAL_ID,Name,Score,Genres,Type,Episodes,Premiered,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,Spring 1998,"In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,Unknown,"other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,26,Spring 1998,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",TV,26,Summer 2002,ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",TV,52,Fall 2004,It is the dark century and the people are suff...


**tien xu ly**

In [44]:
anime.isnull().sum()
anime.dropna(inplace=True)
anime.isnull().sum()
anime.shape
anime.duplicated().sum()

0

**Building**
+ *Name, des* : dùng tf_idf để xác định xem những cụm từ nào có liên quan (hoặc không liên quan) về mặt chủ đề bằng cách phân tích tần suất một cụm từ xuất hiện trên một trang và tần suất ước tính xuất hiện trên một trang trung bình, trong một tập hợp lớn hơn bao gồm nhiều tài liệu
+ *Genres* : dùng Count Freq để chuyển một văn bản nhất định thành một vectơ trên cơ sở tần suất (số lượng (count)) của mỗi từ xuất hiện trong toàn bộ văn bản

In [45]:
# incides dùng để spot tên anime
indices = pd.Series(anime.index, index = anime['Name'])

In [46]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf2 = TfidfVectorizer(stop_words='english')
count = CountVectorizer(stop_words='english')

+ *đưa vào ma trận để tính cosine similarity* 

In [47]:
tfidf_matrix = tfidf.fit_transform(anime['Name'])
count_matrix = count.fit_transform(anime['Genres'])
tfidf2_matrix = tfidf2.fit_transform(anime['sypnopsis'])

In [48]:
name_similarity = cosine_similarity(tfidf_matrix)
genre_similarity = cosine_similarity(count_matrix)
synopsis_similarity = cosine_similarity(tfidf2_matrix)

**tạo hàm**

In [49]:
def get_recommendations(anime):
    i = indices[anime]
    # tạo list
    name_score = list(enumerate(name_similarity[i]))
    genre_score = list(enumerate(genre_similarity[i]))
    synopsis_score = list(enumerate(synopsis_similarity[i]))
    # sắp xếp theo thứ tự tăng dần của chỉ số.
    name_score = sorted(name_score, key=lambda x: x[0])
    genre_score = sorted(genre_score, key=lambda x: x[0])
    synopsis_score = sorted(synopsis_score, key=lambda x: x[0])
    # độ chính xác = avg score
    combined_score = [(i, (sc_1 + sc_2 + sc_3) / 3) for (i, sc_1), (_, sc_2), (_, sc_3) in zip(name_score, genre_score, synopsis_score)]
    # sắp xếp theo độ chính xác
    combined_score = sorted(combined_score, key=lambda x: x[1], reverse=True)
    
    anime_ids = [i[0] for i in combined_score[1:11]]
    anime_recs = []
    # check anime lưu vào anime_recs
    index = 0
    while len(anime_recs) != 10:
        anime_id = combined_score[1:][index][0]
        index += 1
        if anime in indices.iloc[[anime_id]].index[0]:
            continue
        else:
            anime_recs.append(indices.iloc[[anime_id]].index[0])
    # output
    recommendations = pd.DataFrame({
        "Anime": anime_recs,
        "Similarity": [combined_score[i][1] for i in range(1, 11)]
    })

    return recommendations

In [52]:
get_recommendations('Naruto')

,Anime,Similarity
0,Boruto: Jump Festa 2016 Special,0.674166
1,Dragon Ball GT: Gokuu Gaiden! Yuuki no Akashi ...,0.625837
2,Dragon Ball Z: Summer Vacation Special,0.605111
3,Rekka no Honoo,0.601149
4,Dragon Ball Z,0.583702
5,Dragon Ball Kai,0.577870
6,Dragon Ball Z: Atsumare! Gokuu World,0.558173
7,"Dragon Ball Z Movie 15: Fukkatsu no ""F""",0.541824
8,Dragon Ball Super,0.534807
9,Dragon Ball Z Movie 11: Super Senshi Gekiha!! ...,0.532657
